In [45]:
#import libraries

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import cv2
import os

In [46]:
#Configuração dos geradores de dados de treinamento e validação

train = ImageDataGenerator(rescale=1/255) #Normalizar os valores de pixels para o intervalo [0,1]
validation = ImageDataGenerator(rescale=1/255)
test = ImageDataGenerator(rescale=1/255)

In [47]:
#Carregar os dados de treinamento e validação

train_dataset = train.flow_from_directory('Dataset/training/',
                                         target_size = (200,200), #Redimensionar as imagens para 200x200 pixels
                                         batch_size = 3, #numero de pixels por lote
                                         class_mode = 'categorical') #Modo de classificação (categorica)

validation_dataset = train.flow_from_directory('Dataset/validation/',
                                         target_size = (200,200),
                                         batch_size = 3,
                                         class_mode = 'categorical')

test_dataset = test.flow_from_directory('Dataset/testing/',
                                         target_size=(200,200),
                                         batch_size=3,
                                         class_mode='categorical')

Found 3213 images belonging to 3 classes.
Found 1140 images belonging to 3 classes.
Found 1052 images belonging to 3 classes.


In [48]:
#Verificar os indices das classes

train_dataset.class_indices

{'clean': 0, 'litter': 1, 'recycle': 2}

In [49]:
# Definindo o modelo da rede neural (MLP)

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(200, 200, 3)),  # Transforma os dados em um vetor
    tf.keras.layers.Dense(512, activation='relu'),  # Camada densa com ativação ReLU
    tf.keras.layers.Dropout(0.5),  # Adicionando Dropout para reduzir overfitting
    tf.keras.layers.BatchNormalization(),  # Adicionando normalização em lotes
    tf.keras.layers.Dense(256, activation='relu'),  # Camada densa com ativação ReLU
    tf.keras.layers.Dense(128, activation='relu'),  # Adicionando mais uma camada densa
    tf.keras.layers.Dense(3, activation='softmax')  # Camada de saída com ativação softmax (três classes)
])

In [50]:
# Compilando o modelo com uma taxa de aprendizado ajustada

model.compile(optimizer = RMSprop(learning_rate=0.01), #Otimizador RMSprop com taxa de aprendizado de 0.001
             loss = 'binary_crossentropy', #Função de perda para classificação multiclasse
             metrics=['accuracy']) #Métrica para avaliação do modelo

In [51]:
# Treinando o modelo

history = model.fit(train_dataset,
          steps_per_epoch=5,  # Número total de lotes (batches) por época
          epochs=15,  # Número de épocas de treinamento
          validation_data=validation_dataset,  # Dados de validação
          validation_steps=5)  # Número total de lotes (batches) de validação

Epoch 1/15
5/5 [==============================] - 4s 624ms/step - loss: 1.2022 - accuracy: 0.3333 - val_loss: 328.4907 - val_accuracy: 0.4667
Epoch 2/15
5/5 [==============================] - 2s 424ms/step - loss: 0.8641 - accuracy: 0.4667 - val_loss: 128.6798 - val_accuracy: 0.2000
Epoch 3/15
5/5 [==============================] - 2s 473ms/step - loss: 0.5996 - accuracy: 0.5333 - val_loss: 14.2835 - val_accuracy: 0.3333
Epoch 4/15
5/5 [==============================] - 2s 367ms/step - loss: 0.7907 - accuracy: 0.3333 - val_loss: 1.2041 - val_accuracy: 0.4000
Epoch 5/15
5/5 [==============================] - 2s 457ms/step - loss: 0.7502 - accuracy: 0.2000 - val_loss: 1.8402 - val_accuracy: 0.2000
Epoch 6/15
5/5 [==============================] - 2s 425ms/step - loss: 0.6973 - accuracy: 0.2000 - val_loss: 1.9012 - val_accuracy: 0.3333
Epoch 7/15
5/5 [==============================] - 2s 505ms/step - loss: 0.7292 - accuracy: 0.1333 - val_loss: 1.0501 - val_accuracy: 0.1333
Epoch 8/15
5/5 

In [ ]:
# Avaliação no conjunto de teste

y_pred = model.predict(test_dataset)
y_true = test_dataset.classes

117/351 [=========>....................] - ETA: 13s

In [ ]:
# Calculando a matriz de confusão

conf_matrix = confusion_matrix(y_true, y_pred.argmax(axis=1))

In [ ]:
# Calculando outras métricas

accuracy = accuracy_score(y_true, y_pred.argmax(axis=1))
classification_rep = classification_report(y_true, y_pred.argmax(axis=1), zero_division=1)
roc_auc = roc_auc_score(y_true, y_pred, multi_class='ovr')

In [ ]:
# Exibindo os resultados
print("Matriz de Confusão:")
print(conf_matrix)

print("\nAccuracy:", accuracy)
print("\nClassification Report:")
print(classification_rep)
print("\nAUC-ROC:", roc_auc)